# Prepare core llm chain

In [ ]:
import langchain
langchain.debug = False # set to True if  you want to see what the LLM is doing

from langchain.chat_models import AzureChatOpenAI

import dotenv
dotenv.load_dotenv()

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm.predict('Are you ready?')

In [ ]:
from personalizer import *
from response_checker import *
from langchain.prompts.prompt import PromptTemplate
import slates

template = """
{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix"],
    template=template,
)
chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/stories.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    policy = slates.RandomPolicy,
    response_checker = LLMResponseCheckerForSlates(
        llm,
        prompt = ChatPromptTemplate.from_messages(
                [
                    LLMResponseCheckerForSlates.default_system_prompt,
                    HumanMessagePromptTemplate.from_template(
                        '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.'''
                    )]
    ))
)
chain.run({
    'prefix': [f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.'],
    'goal': 'Write a funny story about',
    'context': 'Friends series',
    'suffix': ['Please try to be as funny as possible.', '']})


In [ ]:
from personalizer import *
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix"],
    template=template,
)
chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/stories.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = LLMResponseCheckerForSlates(
        llm,
        prompt = ChatPromptTemplate.from_messages(
                [
                    LLMResponseCheckerForSlates.default_system_prompt,
                    HumanMessagePromptTemplate.from_template(
                        '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.'''
                    )]
    ))
)
chain.run({
    'prefix': Embed([f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.']),
    'goal': Embed('Write a funny story about'),
    'context': 'Friends series',
    'suffix': ['Please try to be as funny as possible.', '']})

# Experiment with mock llm

In [ ]:
from response_checker import *

class MockLLMChain:
    outcomes: List[List[float]] = None
    
    def __init__(self, outcomes, prompt):
        self.outcomes = outcomes
        self.prompt = prompt

    def predict(self, prefix, suffix, **kwargs):
        return str(self.outcomes[int(prefix)][int(suffix)])

class MockChecker(ResponseChecker):
    def __init__(self):
        ...

    def grade_response(
        self, inputs: Dict[str, Any], llm_response: str, **kwargs
    ) -> float:
        return float(llm_response)
    
from personalizer import *
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{context}
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "context", "suffix"],
    template=template,
)
chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = MockChecker()
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)
chain.run({
    'prefix': ['0', '1'],
    'context': 'bla',
    'suffix': ['0', '1']})

In [ ]:
import pandas as pd

chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = MockChecker()
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

result = []
for i in range(1000):
    x = chain.run({
        'prefix': ['0', '1'],
        'context': 'bla',
        'suffix': ['0']})
    result.append(float(x))

pd.DataFrame({'x': result})['x'].rolling(window=10).mean().plot()

In [ ]:
import pandas as pd

chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = MockChecker()
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

result = []
for i in range(1000):
    x = chain.run({
        'prefix': ['0', '1'],
        'context': 'bla',
        'suffix': ['1']})
    result.append(float(x))

pd.DataFrame({'x': result})['x'].rolling(window=10).mean().plot()

In [ ]:
import pandas as pd

chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = MockChecker()
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

result = []
for i in range(1000):
    x = chain.run({
        'prefix': ['0', '1'],
        'context': 'bla',
        'suffix': ['0', '1']})
    result.append(float(x))

pd.DataFrame({'x': result})['x'].rolling(window=10).mean().plot()

In [ ]:
import pandas as pd

chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_logs = 'logs/mock.txt',
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = MockChecker()
)
chain.llm_chain = MockLLMChain([
    [0, 0.3],
    [0.6, 0.9]], prompt = prompt)

result = []
for i in range(1000):
    x = chain.run({
        'prefix': ['0'],
        'context': 'bla',
        'suffix': ['0', '1']})
    result.append(float(x))

pd.DataFrame({'x': result})['x'].rolling(window=10).mean().plot()